## Plan of action
- grab sentence by sentence file
- split each sentence in n-grams (1-2-3-4)
- compare n-grams with dictionary
- if n-gram is contained in any of the dictionaries, grab the whole sentence and mark is as the respective \_\_label\_\_='whatever'
    - if more than one label can be assigned, assign them both
    - if a label is already assigned, assign the next one too and raise a message
- compare sentences, if they contain equivalent labels, merge them into a paragraph and put the label on it. Do until we reach a sentence with a different labels

This will be the training data

Test data will be 2-3 more courses I will find in sourcera and split the same way as my current data, 
will learn a classifier with it and try to classify them

### Whats and Whys
- ***Overall goal***: to create a labeled training set, but not by evaluating it manually more than selecting terms to look for
- ***N-gram extraction***: To help find the dictionary terms
- ***Keywords extraction from title***: to help figure out the main topic of the LO

##### Next steps
* After finding and matching the n-grams as per sentences, I will add the respective label to the end of the sentence with a comma before. 
* Then will split each sentence by these commas and save into a list. 
* Then if a sentence has one item in the list, i.e. only sentence, that means it will have no label. If it has 2 elements in the list, that means it has one label if it has 3 elements - 2 labels etc. 
    * if it has no label - adopt the label that the first sentence above it has, go from that sentence up until a label is found and assign it
        * if the sentence above has one label, adopt it
        * if 2 labels - do nothing
    * if one label - good
    * if 2 labels - manual check and leave one

### Dictionary terms for my 5 classes

In [68]:
conceptdescr_dict = ['this research','refers to','last but not least','number one',
                     'number two','number three','drawback','advantages',
                     'disadvantages','benefits','drawbacks','defined',
                     'there are many types','important to understand',
                     'let\'s look at','when we use the term','imagine that',
                     'refers to as','key concepts','this is a term','more specifically',
                     'helps to','is used to','makes it easier to',
                     'provide information about','when talking about','critical to understand',
                     'let\'s take a look at','what is','crucial','can be used to','ensures',
                     'rule of thumb','consider','last but not least','topic of this',
                     'topic of today','one way to','another possibility is',
                     'next one is','problem','to address','want to do','different ways',
                     'approach is to','tries to','try to','topic is','method','task',
                     'in this way','we specify','by combining','results']

conceptmention_dict = ['called','use the term','more specifically','this includes',
                       'activities surrounding','show you','additionally','should include',
                       'types of','recommends','known as','known for','advantages to',
                       'disadvantages to','benefits','drawbacks','plays a key role',
                       'refers to as','first one is','we introduce','techniques',
                       'second one is','traditional','method','-based']

application_dict = ['best practices','best practice','means that','practical benefit',
                    'sorts of things','good practices','best to','you should','you shouldn\'t',
                    'may find that','can use','this is important','particularly importnant',
                    'is why','useful to','recommended that','consider','useful to']

example_dict = ['example','there are many','many types','include','can see','hands on',
                'includes','exemplar','prototype','sample','case','illustration','analogy',
                'let\'s think of','may want to','some of','most of','different types of',
                'e.g.','for example','key aspects','key concepts','in other words',
                'of these are','of them are','rather than','such as','pros and cons',
                'a long list','various','for instance','it\'s like','different ways',
                'tried to','instead of']

summary_dict = ['by the end of','as you know','have a good understanding','will begin by',
                'will cover','well delve','will discuss','will be able to','will understand',
                'to summarize','summary','take away','to conclude','as you know','introduced',
                'be aware','we\'ve looked at','last week we discussed','discuss how to',
                'will describe','the goal']

### Import all modules

In [69]:
# Import all necessary modules for EVERYTHING here
import os
import sys
import os.path
import string
import time

import math
from textblob import TextBlob as tb
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.util import ngrams

import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from tabulate import tabulate

### Playground

In [70]:
# 1,2,3,4 grams

"""
search for part of string in a string as:

searchFor = 'THIS'.lower()
for item in conceptmention_dict:
    if searchFor in item:
        print("found \"{}\" in \"{}\"".format(searchFor,item))
   
just as a caution, if you have a string "paratyphoid is bad" 
and you do a if "typhoid" in "paratyphoid is bad" you will get a true
"""

searchFor = 'based'.lower()

for item in conceptmention_dict:
    if searchFor in item:
        print(item+" __label__=example")

-based __label__=example


## Trying to find n-grams in the dictionary

- Very unsure with monograms - lots of matches with simple stopwords, some of them make sense, but very few.
- Very little found with trigrams, but if found - it's good
- Bigrams should work good, but not too good
- Tetragrams give no result whatsoever, also because there are very few items in the dictionaries that have 4 words to match with

#### Dummy text to play with

In [71]:
#### WITHOUT STOP WORDS

textnoSW = """music hello 'm good to see back our course 
before start with this video let me remind what has been covered earlier videos 
by should have an idea of what an embedded processor how it works 
should also be familiar with features of an embedded processor 
must remind again this course make emphasis on microcontrollers 
therefore large field of embedded processors reduced to only microcontrollers 
after learn about microcontrollers next step would be to somehow put your hands on them start actually working with them 
one way to do so to buy microcontroller breadboard set of resistors capacitors wires whatever else necessary to build proper electrical circuits assemble all circuits by yourself on breadboard 
well it does n't sound cool because it 's long process 
know there always one resistor need missing end there will be little motivation to continue playing with microcontrollers 
luckily there another option to buy ready made platform with mounted microcontroller to start working with it immediately 
this video 're going to talk about these platforms 
let 's start with arduino 
arduino widely known platform for fast prototyping 
platform uses atmel microcontrollers at its core 
overall there are about 24 boards available 
on website can find them organized according to application areas such as internet of things wearable others 
besides boards there are many shields built specifically for our arduino devices 
functionality of these shields vary lot 
for example with an mp3 shield can build your own mp3 player if need one 
wi-fi shield great tool to build applications let 's say domain of internet of things 
joystick shield suitable for control purposes 
so if want to extend functionality of board your application can plug shield 
luckily it can easily be done thanks to sockets installed arduino 
arduino also provides its own integrated development environment for software development which easy to use really speeds up development process 
second platform would like to mention stm nucleo platform 
at its core there are stx complex cortex microcontrollers from m0 to m4 
so can choose here between speed energy consumption 
boards also have extension connectors connectors are compatible with arduino shields which very convenient 
also there so-called morpho connector for other kind of shields 
very convenient feature of this platform programmer/debugger mounted on board 
there an online id available for this platform as well as whole set of tools including operating system tools developer ecosystem for building internet of things solutions 
offline ides are also available 
they are more complex than arduino 's one but they give more freedom to developer 
would say this platform for more experienced users 
here mentioned two kinds of general purpose platforms can be extended with help of shields to target some specific areas 
there are of course more platforms available on market 
some of them target one specific area might not be extensible 
so there legit question might have 
how do select certain platform from this diversity 
first of all choice obviously should be application-oriented 
for example if want to do some digital signal processing might want to use microcontroller supports some dsp operations 
if need high computational power then cortex m4 or cortex m3 microcontrollers would suit well since they have highest clock frequency among previous mentioned mcus 
therefore your choice would be either arduino due or stm nuclear cortex m4 m3 
if small size crucial for there no need of high performance then probably want to check arduino micro 
it also important to consider platforms are interchangeable sense same application can be implemented on two different platforms 
something would like to emphasize community crucial when it comes to choosing platform 
bigger community more likely problems might face have already been solved by someone else which case can probably find answer internet 
would like to show real project was implemented using microcontrollers 
it metering system 
imagine have tube there some liquid flowing through tube 
goal of system to measure liquid 's flow speed 
since know size of tube need to calculate volume of liquid during time interval 
finally need to store those measurements memory 
what more device should have user interface like keyboard small screen 
so where to start 
well start by dividing tasks two components metering unit control unit 
next thing to do would be to decide how to measure liquid flow speed 
this can be done using ultrasound 
need to emit an ultrasonic sound receive it back do some spectral analysis to estimate liquid flow speed 
do n't worry if do n't understand what spectral analysis 
here what want to show approach 
so need to do some digital signal processing to get liquid flow speed 
after get those measurements do some simple averaging finally store those processed measurements 
okay so have set of tasks signal processing averaging storing user interface 
it would make sense to map set of tasks onto two microcontrollers 
let 's say signal processing will be done on mcu 1 will call this part metering unit 
other tasks will run on control unit which will be implemented on mcu2 
both parts will be connected together using uart interface 
since mcu1 does spectral analysis cortex m4 microcontroller would suit well since it has dsp extensions 
will also need an adc to sample ultrasonic signal 
on other hand mcu2 does not need to support dsp operations 
but it need to be fast enough to communicate with screen keyboard while doing averaging at same time 
it should have enough pins to work with keyboard at least two uart units to work with screen to communicate with metering unit 
cortex m3 would fit well 
it useful to leave some room for expansion 
this means might want to have some extra functionality to device future 
therefore selected microcontrollers should have enough resources terms of performance memory 
finally it important to notice system can be implemented many other ways 
this video discussed ways on how to actually start working with microcontrollers using platforms 
hopefully have an idea of how to pick an appropriate platform for your application 
good luck 
sound """

## N-GRAM IMPLEMENTATION

In [ ]:
#### WORKS

def get_ngrams(text, n):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

def find_grams_inDict1(gramsList,sentenceList):
    sentences = sentenceList
    
    for gram in gramsList:
        for item in conceptdescr_dict:
            if gram in item:
                print("========= FOUND \"{}\" IN \"{}\" \t|LABEL|\t _ConceptDescription_".format(gram, item))
                """
                for sent in sentences:
                    if gram in sent:
                        print("\" "+sent+"\"\n")
                """
        for item in example_dict:
            if gram in item:
                print("========= FOUND \"{}\" IN \"{}\" \t|LABEL|\t _Example_".format(gram, item))
                """
                for sent in sentences:
                    if gram in sent:
                        print("\" "+sent+"\"\n")
                """
        for item in conceptmention_dict:
            if gram in item:
                print("========= FOUND \"{}\" IN \"{}\" \t|LABEL|\t _ConceptMention_".format(gram, item))
                """
                for sent in sentences:
                    if gram in sent:
                        print("\" "+sent+"\"\n")
                """
        for item in application_dict:
            if gram in item:
                print("========= FOUND \"{}\" IN \"{}\" \t|LABEL|\t _Application_".format(gram, item))
                """
                for sent in sentences:
                    if gram in sent:
                        print("\" "+sent+"\"\n")
                """
        for item in summary_dict:
            if gram in item:
                print("========= FOUND \"{}\" IN \"{}\" \t|LABEL|\t _Summary_".format(gram, item))
                """
                for sent in sentences:
                    if gram in sent:
                        print("\" "+sent+"\"\n")
                """

                
def find_grams_inDict2(gramsList1,gramsList2,sentenceList):
    print("_____________________________________________________________________\n")
    find_grams_inDict1(gramsList1,sentenceList)
    print("\n_____________________________________________________________________\n")
    find_grams_inDict1(gramsList2,sentenceList)
    

def find_grams_inDict3(gramsList1,gramsList2,gramsList3,sentenceList):
    print("_____________________________________________________________________\n")
    find_grams_inDict1(gramsList1,sentenceList)
    print("\n_____________________________________________________________________\n")
    find_grams_inDict1(gramsList2,sentenceList)
    print("\n_____________________________________________________________________\n")
    find_grams_inDict1(gramsList3,sentenceList)

"""
sentences = textnoSW.split("\n")
MonoGrams = get_ngrams(textnoSW,1)
BiGrams = get_ngrams(textnoSW,2)
TriGrams = get_ngrams(textnoSW,3)
TetraGrams = get_ngrams(textnoSW,4)    # nothing.. senseless

#find_grams_inDict1(MonoGrams, sentences)
find_grams_inDict2(BiGrams, TriGrams, sentences)
#find_grams_inDict3(MonoGrams, BiGrams, TriGrams, sentences)
"""

## Extracting N-Grams and matching title keywords to real files

In [73]:
def title_extrKeys(curFilePath):
    titleinx = len(curFilePath.split("\\"))-1
    fullTitle = curFilePath.split("\\")[titleinx]
    
    # remove all surrounding stuff like my naming convention etc from the title 
    mainTitlelist = fullTitle.split("_")
    maintitle = mainTitlelist[1]
    punct = {'_','-','.'}
    finaltitle = ""
    
    # extract the final title, i.e. the main part of the title
    for word in maintitle.split():
        for letter in word:
            if letter in punct:
                finaltitle += " "
                pass
            else:
                finaltitle += letter
    
    title_keywords = []
    for word in finaltitle.split(" "):
        if word == 'en':
             pass
        else:
            title_keywords.append(word)
    
    return title_keywords  # returns a list of keywords

In [95]:
path = r"C:\Users\ani\Desktop\Course data Thesis\one file"

for root, subdirs, files in os.walk(path):

    for curFile in os.listdir(root):

        curFilePath = os.path.join(root, curFile)

        if os.path.isdir(curFilePath):
            pass

        else:
            if not curFilePath.endswith("_sentNoPunctNoSWTiny.txt"):
                pass
            else: 
                curFile = open(curFilePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK
                oFilePath = str(curFilePath.split(".en")[0])+".en_BoWv2Top50.txt"
                
                # extract keywords from the title of the file
                title_keywords = title_extrKeys(curFilePath)
                
                # convert the file content into a string variable holding everything there
                sentences = curFile.read().split("\n")
                text = ""
                for sent in sentences:
                    text += sent+"\n"
                    
                # look for title keywords within the text
                for keyword in title_keywords:
                    for sent in sentences:
                        if keyword in sent:
                            outp = str("<<"+keyword+">> FOUND IN <<"+sent+">>").strip()
                            print(outp.strip())
                    
                # extract n-grams
                MonoGrams = get_ngrams(text,1)
                BiGrams = get_ngrams(text,2)
                TriGrams = get_ngrams(text,3)

                # look for grams in text
                find_grams_inDict2(BiGrams, TriGrams, sentences)
                

<<behaviors>> FOUND IN <<so first lecture 're going return this idea of behaviors >>
<<behaviors>> FOUND IN <<but actually talked about behaviors before >>
<<behaviors>> FOUND IN <<so this really what need do need revisit behaviors context of control failure >>
<<behaviors>> FOUND IN <<these are key behaviors always need >>
<<behaviors>> FOUND IN <<so our job design these two behaviors using what 've already learned >>
<<behaviors>> FOUND IN <<well solution again make dependent on or actually distances so normal e. aviod this being overly cautious are actually going switch between behaviors fact what 're going do using something like induced mode sliding mode very gracefully combine goal goal avoid obstacles but for let me just point out one clever thing for instances say want care more about obstacle closer get so want be bigger closer obstacle get >>
<<behaviors>> FOUND IN <<par- particular 've seen these 2 key behaviors goal goal avoid obstacles >>
__________________________________